In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('TaxiCalifornia.csv')


In [3]:
df.head()


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,26:21.0,5.8,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [4]:
df.passenger_count.describe()

count    27246.000000
mean         1.666300
std          1.288571
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max          6.000000
Name: passenger_count, dtype: float64

## Our end feature that is price will be Normally distributed

## we are imputing mean values where the values were not assigned

In [5]:
df['dropoff_latitude'].replace(to_replace = 0.0 , value = df['dropoff_latitude'].mean() , inplace = True)

In [6]:
df['pickup_latitude'].replace(to_replace = 0.0 , value = df['pickup_latitude'].mean() , inplace=True)

In [7]:
df['pickup_longitude'].replace(to_replace = 0.0 , value = df['pickup_longitude'].mean() , inplace = True)

In [8]:
df['dropoff_longitude'].replace(to_replace = 0.0 , value = df['dropoff_longitude'].mean() , inplace = True)

## NOW GETTING DISTANCE FROM LONGITUDE AND LATITUDE


Formula for conversion in Latitude

 1 degree  = 111.2 km 
 
Formula for conversion in Longitude

 1 degree  = 111.2 + cos(latitude)
 
 ### So we are going to create new feature of distance and after that we will drop the Co ordinates

In [9]:
## latitude

degree_lat = abs(df['dropoff_latitude'] - df['pickup_latitude'])



In [10]:
## Longitude
import math



degree_long  = abs(df['dropoff_longitude'] - df['pickup_longitude'])

##dist_long = degree_long*111.2 + math.cos(degree_long).astype(float)

## distance = math.sqrt((degree_lat*111.2)**2 +(degree_long*111.2*np.cos(degree_lat))**2)

In [11]:
degree_lat

0        0.009041
1        0.070701
2        0.010708
3        0.024949
4        0.015754
           ...   
27241    0.001910
27242    0.010239
27243    0.022562
27244    0.022583
27245    0.027314
Length: 27246, dtype: float64

In [12]:
cos_value = np.cos(degree_lat)

In [13]:
distance =  np.sqrt((degree_lat*111.2)**2 +(degree_long*111.2*cos_value)**2)

In [14]:
distance

0        1.049262
1        8.857448
2        1.520519
3        2.817834
4        2.164918
           ...   
27241    1.543899
27242    1.739995
27243    4.052173
27244    3.123064
27245    3.251904
Length: 27246, dtype: float64

In [15]:
df['distance'] = distance

In [16]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,26:21.0,5.8,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,1.049262
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,8.857448
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,1.520519
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2.817834
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2.164918


In [17]:
## Finally we inserted the distance value the we have to drop our Co ordinates values

df.drop({'key' , 'pickup_datetime' , 'pickup_longitude' , 'pickup_latitude' , 'dropoff_latitude', 'dropoff_longitude'} , axis = 1 , inplace  =True)

In [18]:
df.head()

,fare_amount,passenger_count,distance
0,5.8,1,1.049262
1,16.9,1,8.857448
2,5.7,2,1.520519
3,7.7,1,2.817834
4,5.3,1,2.164918


This demonstrates that our data has some outliers , But we can deal with it Using Optimal machine learning algorithms

This implies that 1 number of passenger are maximum in the dataset

In [19]:
X = df.iloc[:,1:]

In [20]:
X

,passenger_count,distance
0,1,1.049262
1,1,8.857448
2,2,1.520519
3,1,2.817834
4,1,2.164918
...,...,...
27241,5,1.543899
27242,1,1.739995
27243,1,4.052173
27244,1,3.123064


In [21]:
y = df.iloc[:,:1]

In [22]:
y

,fare_amount
0,5.8
1,16.9
2,5.7
3,7.7
4,5.3
...,...
27241,6.0
27242,7.3
27243,10.9
27244,9.0


## Now Applying different machine learning algorithms along with hyperparameter tuning

In [23]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

### Since we know Linear regressions are prone to features and outliers so we to perform some feature scaling 

In [24]:
from sklearn.preprocessing import MinMaxScaler
MMS = MinMaxScaler()

  ### WE are dividing the data into training and testing for supervised learning

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test  = train_test_split(X,y,test_size = 0.25 , random_state = 1)

In [26]:
X_train

,passenger_count,distance
21900,1,5.205731
3465,1,1.134409
24071,1,2.577150
4077,2,2.516324
14501,1,2.272095
...,...,...
10955,1,1.047250
17289,4,22.482930
5192,1,3.205788
12172,1,2.516762


### perform feature scaling

In [27]:
X_train = MMS.fit_transform(X_train)

In [28]:
X_train

array([[1.66666667e-01, 1.29929375e-04],
       [1.66666667e-01, 2.83136083e-05],
       [1.66666667e-01, 6.43228563e-05],
       ...,
       [1.66666667e-01, 8.00129673e-05],
       [1.66666667e-01, 6.28156364e-05],
       [8.33333333e-01, 5.14265651e-05]])

In [29]:
X_test

,passenger_count,distance
1362,6,0.306555
7996,2,3.570186
1853,1,1.448478
9742,1,1.134409
543,5,3.152289
...,...,...
11437,2,0.000000
17524,1,2.785897
23363,1,1.524470
8360,2,21.532479


In [30]:
X_test = MMS.transform(X_test)

In [31]:
X_test

array([[1.00000000e+00, 7.65128169e-06],
       [3.33333333e-01, 8.91079503e-05],
       [1.66666667e-01, 3.61524357e-05],
       ...,
       [1.66666667e-01, 3.80491130e-05],
       [3.33333333e-01, 5.37427227e-04],
       [1.66666667e-01, 2.91499451e-05]])

 ## Now fitting our data into Linear regression Model

In [32]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
y_pred = lr.predict(X_test)

In [34]:
from sklearn.metrics import r2_score , mean_squared_error 

In [35]:
score_value = r2_score(y_test , y_pred)


In [36]:
square_error = mean_squared_error(y_test,y_pred)

In [37]:
score_value

-0.00264619092325602

In [38]:
square_error

81.94227059359882

#### This clearly shows that our model is not good

 ## Now trying with ensemble techniques

In [39]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

In [40]:
rfr.fit(X_train , y_train)

C:\Users\AASHU\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [41]:
rfr_pred = rfr.predict(X_test)

In [42]:
from sklearn.metrics import r2_score

In [43]:
score2 = r2_score(y_test , rfr_pred)

In [44]:
score2

0.7036257840458047

In [45]:
square_error_rfr = mean_squared_error(y_test , rfr_pred)

In [46]:
square_error_rfr

24.22148153609573

## We can also tune our model to increase our chances of getting right output

In [47]:
rfr_pred

array([ 8.13298023, 13.30675   ,  6.87232326, ...,  6.87232326,
       53.8301    ,  6.85056902])

In [48]:
rfr_pred.shape

(6812,)

In [49]:
y_test

,fare_amount
1362,3.5
7996,9.3
1853,6.5
9742,13.5
543,11.3
...,...
11437,5.3
17524,10.5
23363,6.0
8360,36.1


## We are dumping our model to read in our another Python app file

In [50]:
import pickle

In [54]:
pickle.dump(rfr, open('model_rfr.pkl','wb')) ## we are dumping random forest regressor file in our disc